# **<p style="text-align: center;">Aprendizaje por refuerzo - Ingeniería en Inteligencia Artificial</p>**
## **<p style="text-align: center;">Trabajo práctico Final - SAC</p>**

#### <p style="text-align: center;">Pettinari Fausto, Schuemer Ignacio, Torres Santiago </p>
#### <p style="text-align: center;">Profesores: Claudio Pose, Gabriel Torre, Nicolás Romero, Tomás Chimenti</p>

# **Quadratic Action Reward Enviroment - *Entorno de Prueba***

In [1]:
import gymnasium as gym
from envs import QuadraticActionRewardEnv

In [2]:
gym.register(
    id="gymnasium_env/QuadraticActionRewardEnv-v0",
    entry_point=QuadraticActionRewardEnv
)
env = gym.make("gymnasium_env/QuadraticActionRewardEnv-v0")

#### **Logging & Debugging (Random Agent)**

In [ ]:
from torch.utils.tensorboard import SummaryWriter
import sys
import os
sys.path.append(os.path.join('..'))
from src.random_agent import random_agent_loop

In [ ]:
seed = 42
writer = SummaryWriter(f"../runs/QuadraticActionRewardEnv")
num_episodes = 5000
random_agent_loop(env, num_episodes, writer, seed)



2025-11-20 20:07:10.942520: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-20 20:07:10.979505: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-20 20:07:12.026217: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Episode 0: avg batch reward = -0.472
Episode 10: avg batch reward = -0.821
Episode 20: avg batch reward = -0.615
Episode 30: avg batch reward = -0.629
Episode 40: avg batch reward = -0.274
Episode 50: avg batch reward = -0.804
Episode 60: avg batch reward = -0.495
Episode 70: avg batch reward = -0.679
Episode 80: avg batch reward = -0.501
Episode 90: avg batch reward = -0.402
Episode 100: avg batch reward = -0.445
Episode 110: avg batch reward = -0.494
Episode 120: avg batch reward = -0.578
Episode 130: avg batch reward = -0.454
Episode 140: avg batch reward = -0.414
Episode 150: avg batch reward = -0.564
Episode 160: avg batch reward = -0.360
Episode 170: avg batch reward = -0.337
Episode 180: avg batch reward = -0.843
Episode 190: avg batch reward = -0.592
Episode 200: avg batch reward = -0.658
Episode 210: avg batch reward = -0.746
Episode 220: avg batch reward = -0.453
Episode 230: avg batch reward = -0.289
Episode 240: avg batch reward = -0.488
Episode 250: avg batch reward = -0.7

## **SAC Agent**

In [ ]:
from src.SAC import SAC
from hyperparameters import SACConfig